In [183]:
import requests
import config
import json
from pprint import pprint
import matplotlib.pyplot as plt
import pandas as pd

In [184]:
base_url='https://www.bootcampspot.com/api/instructor/v1'
login={"email": config.email,
       "password": config.password}

### login

In [235]:
response=requests.post(base_url+'/login', json=login)
response_json=response.json()
auth_token=response_json['authenticationInfo']['authToken']

### me

In [237]:
headers={'Content-Type': 'application/json', 'authToken': auth_token}
response=requests.get(base_url+'/me', headers=headers)
response_json=response.json()
# display(pprint(response_json))
courses=response_json['Enrollments']
print('Available Courses: ')
# for each_course in courses: 
#     print(each_course['course']['name'])

Available Courses: 


In [222]:
def get_enrollmentId_courseId(course):
    for each_course in courses: 
        if each_course['course']['name']==course: 
            return (each_course['id'], each_course['courseId'])
    return (None, None)
enrollmentId, courseId=get_enrollmentId_courseId('DU-DEN-DATA-PT-12-2020-U-C-TTH')
# enrollmentId, courseId=get_enrollmentId_courseId('DU-DEN-DATA-PT-08-2020-U-C-MW')

### grades

In [236]:
headers={'Content-Type': 'application/json', 'authToken': auth_token}
payload={'courseId': courseId}
response=requests.post(base_url+'/grades', headers=headers, json=payload)
response_json=response.json()
grades_df=pd.DataFrame(response_json)

### assignments

In [224]:
headers={'Content-Type': 'application/json', 'authToken': auth_token}
payload={'enrollmentId': enrollmentId}
response=requests.post(base_url+'/assignments', headers=headers, json=payload)
response_json=response.json()
current_week_assignments=response_json['currentWeekAssignments']
calendar_assignments=response_json['calendarAssignments']
required_assignments=[each_assignment for each_assignment in calendar_assignments if each_assignment['required']==True]
required_assignments_df=pd.DataFrame(required_assignments)
required_assignments_df.head()

,id,courseId,contextId,assignmentDate,dueDate,bufferDays,effectiveDueDate,title,required,requiredForGraduation,assignmentHeader,assignmentContent,assignmentHours,context,year,month,category
0,89721,2930,1,2020-12-01T07:00:00Z,2020-12-13T06:59:59Z,0,2020-12-13T06:59:59Z,1. Excel Homework,True,False,"{'id': 89585, 'assignmentId': 89721, 'header':...",None,None,"{'id': 1, 'contextCode': 'academic', 'name': '...",0,0,"{'code': 'academic', 'name': 'Academic'}"
1,89713,2930,1,2020-12-08T07:00:00Z,2020-12-20T06:59:59Z,0,2020-12-20T06:59:59Z,2. Visual Basic Homework,True,False,"{'id': 89577, 'assignmentId': 89713, 'header':...",None,None,"{'id': 1, 'contextCode': 'academic', 'name': '...",0,0,"{'code': 'academic', 'name': 'Academic'}"
2,89708,2930,1,2020-12-15T07:00:00Z,2021-01-08T06:59:59Z,2,2021-01-10T06:59:59Z,3. Python Homework,True,False,"{'id': 89572, 'assignmentId': 89708, 'header':...",None,None,"{'id': 1, 'contextCode': 'academic', 'name': '...",0,0,"{'code': 'academic', 'name': 'Academic'}"
3,89706,2930,1,2021-01-05T07:00:00Z,2021-01-17T06:59:59Z,0,2021-01-17T06:59:59Z,4. Pandas Homework,True,False,"{'id': 89570, 'assignmentId': 89706, 'header':...",None,None,"{'id': 1, 'contextCode': 'academic', 'name': '...",0,0,"{'code': 'academic', 'name': 'Academic'}"
4,89726,2930,1,2021-01-12T07:00:00Z,2021-01-24T06:59:59Z,0,2021-01-24T06:59:59Z,5. Matplotlib Homework,True,False,"{'id': 89590, 'assignmentId': 89726, 'header':...",None,None,"{'id': 1, 'contextCode': 'academic', 'name': '...",0,0,"{'code': 'academic', 'name': 'Academic'}"


### assignment_details

In [205]:
assignmentId=int(required_assignments_df['id'][0])
assignmentId

89721

In [240]:
headers={'Content-Type': 'application/json', 'authToken': auth_token}
data=[]

for idx, each_assignment in required_assignments_df['id'].iteritems(): 
    print(each_assignment)
    payload={'assignmentId': int(each_assignment)}
    response=requests.post(base_url+'/assignmentDetail', headers=headers, json=payload)

    response_json=response.json()
    assignment=response_json['assignment']
    students=response_json['students']
    
    for each_student in students: 
        assignment_dict={}
        good_record=False
        try: 
            assignment_dict['assignment_due_date']=assignment['dueDate']
            assignment_dict['assignment_header']=assignment['assignmentHeader']['header']
            assignment_dict['student']=each_student['student']['firstName']+' '+each_student['student']['lastName']
            assignment_dict['submission_date']=each_student['submission']['date']
            assignment_dict['grade']=each_student['grade']['grade']
            assignment_dict['grade_date']=each_student['grade']['date']
            good_record=True
        except: 
            pass
#             print(f"No submission for {each_student['student']['firstName']} for {assignment['assignmentHeader']['header']}")
        if good_record: 
                  data.append(assignment_dict)

assignment_detail_df=pd.DataFrame(data)
# assignment_detail_df.head()

89721
89713
89708
89706
89726
89723
89720
89717
89718
89715
89705
89707
89725
89722
89719
89716
89714
89712
89709


In [241]:
pd.DataFrame(assignment_detail_df.groupby('assignment_header').size(), columns=['Count'])

,Count
assignment_header,
1. Excel Homework,40
10. SQLAlchemy Homework,34
11. HTML and CSS Homework,29
12. Web Scraping and Mongo Homework,16
2. Visual Basic Homework,37
3. Python Homework,38
4. Pandas Homework,37
5. Matplotlib Homework,36
6. Python APIs Homework,32


In [242]:
# assignment_detail_df[assignment_detail_df['assignment_header']=='11. HTML and CSS Homework']['student']

In [243]:
# pd.DataFrame(assignment_detail_df.groupby('student').size(), columns=['Count']).sort_values(by='Count')

In [229]:
# assignment_detail_df['grade_date']=pd.to_datetime(assignment_detail_df['grade_date'])

In [230]:
# assignment_detail_df

,assignment_due_date,assignment_header,student,submission_date,grade,grade_date
0,2020-12-13T06:59:59Z,1. Excel Homework,Aishwarya rao Ala,2020-12-12T02:34:37.782926Z,A,2020-12-16 04:27:37.475480+00:00
1,2020-12-13T06:59:59Z,1. Excel Homework,Alyson Amtman,2020-12-08T00:23:40.614206Z,A,2020-12-16 03:27:38.872976+00:00
2,2020-12-13T06:59:59Z,1. Excel Homework,Andre Shearer,2020-12-12T01:26:27.883387Z,A+,2020-12-16 04:27:47.810325+00:00
3,2020-12-13T06:59:59Z,1. Excel Homework,Anna Kantor,2020-12-12T04:44:15.970044Z,A+,2020-12-16 17:28:01.193589+00:00
4,2020-12-13T06:59:59Z,1. Excel Homework,Anna Sours,2020-12-11T23:08:36.207606Z,A,2020-12-16 02:28:17.102561+00:00
...,...,...,...,...,...,...
350,2021-03-07T06:59:59Z,11. HTML and CSS Homework,Swati Dham,2021-03-07T06:46:16.009791Z,A+,2021-03-09 16:08:22.408729+00:00
351,2021-03-07T06:59:59Z,11. HTML and CSS Homework,TJ Ossola,2021-03-06T20:10:43.342816Z,A+,2021-03-09 18:08:11.025031+00:00
352,2021-03-07T06:59:59Z,11. HTML and CSS Homework,Tricia Harris,2021-03-06T04:26:53.704045Z,A,2021-03-09 22:09:13.243059+00:00
353,2021-03-07T06:59:59Z,11. HTML and CSS Homework,Verity Fiadonu,2021-03-06T22:16:30.322704Z,A+,2021-03-10 00:08:18.007096+00:00


In [231]:
assignment_detail_df['grade'].value_counts()

A+            176
A             115
A-             21
B+             17
B              10
I               4
C+              3
Incomplete      2
C               2
B-              2
D               1
F               1
D-              1
Name: grade, dtype: int64

In [232]:
df['grade'].value_counts()

A+            175
A             117
A-             21
B+             17
B              10
I               4
C+              3
C               2
Incomplete      2
B-              2
D               1
F               1
D-              1
Name: grade, dtype: int64